In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from sklearn.metrics import mean_squared_error
import os
import sys
import pickle
import itertools

sys.path.append('../Models')
sys.path.append('../')

import wrmf, wrmf_helpers
from settings import PATH_TO_SPARSE_MATRIX

# (1) Compute the wrmf factors 

In [2]:
"""
NOTE: ONLY RUN THIS IF YOU WANT TO RECOMPUTE WRMF WITH NEW PARAMS.
OTHERWISE USE PRECOMPUTED FACTORIZATION (STORED LOCALLY/IN DRIVE)
"""
# params = {
#     'factors': 20,
#     'reg': 0.1,
#     'iters': 20,
#     'alpha': 15
# }
# song_factors = wrmf_helpers.get_fitted_wrmf(PATH_TO_SPARSE_MATRIX, params)

'\nNOTE: ONLY RUN THIS IF YOU WANT TO RECOMPUTE WRMF WITH NEW PARAMS.\nOTHERWISE USE PRECOMPUTED FACTORIZATION (STORED LOCALLY/IN DRIVE)\n'

# (2) Get the song factorization 

In [3]:
with open('wrmf_song_factors.pickle', 'rb') as f:
    song_factors = pickle.load(f)

# (3) Get "n_similar" top songs from wrmf 

In [4]:
# example: It Wasn't Me, by Shaggy and Ignition - Remix
song_ids = ['1lzr43nnXAijIGYnCT8M8H', '5dNfHmqgr128gMY2tc5CeJ']
top_matches = wrmf_helpers.get_top_tracks(song_factors, song_ids, n_similar=10, verbose=True)


Recommended Songs for ["It Wasn't Me", 'Ignition - Remix']
------------------------------------------------------------
Track Name                                        Score
------------------------------------------------------------
Ignition - Remix                                  0.990172
It Wasn't Me                                      0.988927
Ride Wit Me                                       0.954857
Gold Digger                                       0.932842
Buy U a Drank (Shawty Snappin') (feat. Yung Joc)  0.930893
Yeah!                                             0.928637
Ms. Jackson                                       0.926211
It Wasn't Me                                      0.924964
Hey Ya!                                           0.922270
Promiscuous                                       0.921835
